Fourier Neural Operator Code
The goal of this notebook is to implement an FNO for a test dataset. Eventually this code will be used for decoding turbulence simulations

Data Preparation


In [1]:
import torch
import jax.numpy as jnp
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, TensorDataset

In [2]:
# Simulated dataset generation
# Inputs: Evolved states (e.g., velocity/pressure fields over time)
# Targets: Forcing functions (spatially/temporally varying fields)


# Example:
data_path = "/Users/carsonmcvay/desktop/gradschool/research/turbulence_encryption/forcing_functions/multi_forcing_simulations.npz"
data = jnp.load(data_path)
inputs = data["inputs_forcing"]
outputs = data["outputs_forcing"]


# # Normalize data
inputs_normalized = (inputs - jnp.mean(inputs))/jnp.std(inputs)


KeyError: 'inputs_forcing is not a file in the archive'

Split training and test data

In [12]:
X_train, X_test, y_train, y_test = train_test_split(inputs_normalized, forcing, test_size=0.2, random_state=42)

Pass data to the FNO

In [14]:
# Convert data to PyTorch tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32)

# Create data loaders
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

TypeError: len() of unsized object

In [9]:
# FNO Architecture
import torch
import torch.nn as nn
import torch.fft

class FNO2D(nn.Module):
    def __init__(self, modes, width):
        """
        Args: 
        modes: number of Fourier modes to retain
        width: Feature width for the network
        """
        super(FNO2D, self).__init__()
        self.modes = modes
        self.width = width

        # input and output transforations
        self.input_layer = nn.Linear(inputs_normalized, width)
        self.output_layer = nn.Linear(width, forcing)

        # Fourier layers
        self.fourier_layers = nn.ModuleList([
            FourierLayer(modes,width) for i in range(4)
        ])

        # Non linearity
        self.activation = nn.ReLU()
    
    def forward(self, x):
        # input transformation
        x = self.input_layer(x) # (batch, height, width, width)

        # pass through Fourier layers
        for layer in self.fourier_layers:
            x = layer(x)
            x = self.activation(x)

        # output transformation
        x = self.output_layer(x) # (batch, height, width, forcing_channels)
        return x

class FourierLayer(nn.Module):
    def __init__(self, modes, width):
        super(FourierLayer, self).__init__()
        self.modes = modes
        self.width = width

        # Learnable Foureier filters
        self.weights = nn.Parameter(torch.randn(width, modes, modes, dtype=torch.cfloat))

    def forward(self, x):
        # Fourer transform of input
        x_ft = torch.fft.fft2(x, dim=(-2, -1)) # FFT over spatial dimensions

        # Retain only the first modes Fourier modes
        x_ft = x_ft[...,:self.modes, :self.modes] * self.weights

        # Inverse fft
        x = torch.fft.ifft2(x_ft, dim=(-2,-1)).real
        return x

In [ ]:
# Training Loop

# initialize model, optimizer, and loss function
fno_model = FNO2D(modes=16, width=64)
optimizer = torch.optim.Adam(fno_model.parameters(), lr=1e-3)
loss_fn = nn.MSELoss()

# training loop
num_epochs = 100
for epoch in range(num_epochs):
    for evolved_state, forcing_function in data_loader: #assuming a data loader
        optimizer.zero_grad()

        # forward pass
        predicted_forcing = fno_model(evolved_state)

        # Compute loss
        loss = loss_fn(predicted_forcing, forcing_function)

        # Backward pass and optimization
        loss.backward()
        optimizer.step()

    print(f"Epoch {epoch + 1}/{num_epochs}, loss: {loss.item()}")



In [ ]:
# Evaluation

# test on unseen data
with torch.no_grad():
    for evolved_state, true_forcing in test_loader:
        predicted_forcing = fno_model(evolved_state)

        # compute error or visualize
        error = loss_fn(predicted_forcing, true_forcing)
        print(f"Test Error: {error.item()}")